# Pandas Usage Example

This provides an minimal usage example for `pydantic-cereal` with Pandas.
To start, use the following imports and create a global `cereal` object:

In [1]:
"""Minimal imports."""

import pandas as pd
from upath import UPath
from pydantic import BaseModel, ConfigDict
from pydantic_cereal import Cereal
from pydantic_cereal.examples.ex_pd import pd_read, pd_write

cereal = Cereal()  # global variable

We must add reader and writer classes for it. 
These must accept [`fsspec`](https://filesystem-spec.readthedocs.io/en/latest/) file system
and path (within that filesystem) as inputs.
We can register these with our `cereal` object by creating a wrapped (`Annotated`) type.

In [2]:
MyDF = cereal.wrap_type(pd.DataFrame, reader=pd_read, writer=pd_write)

We can use this type in a Pydantic model:

In [3]:
class ExampleModel(BaseModel):
    """Example model."""

    model_config = ConfigDict(arbitrary_types_allowed=True)

    df: MyDF  # NOTE: Make sure to use the wrapped type!
    value: str = "default_value"

You can instantiate objects as usual:

In [4]:
mdl = ExampleModel(df=pd.DataFrame({"foo": ["a", "b", "c"], "bar": [1, 2, 3]}))

Now, you can write your model to an arbitrary directory-like `fsspec` URI.
In this example, we're writing to a temporary [`MemoryFileSystem`](https://filesystem-spec.readthedocs.io/en/latest/api.html#fsspec.implementations.memory.MemoryFileSystem):

In [5]:
cereal.write_model(mdl, "memory://example_model")

'/example_model'

And we can load another object from there:

In [6]:
obj = cereal.read_model("memory://example_model")
assert isinstance(obj, ExampleModel)
obj.df

,foo,bar
0,a,1
1,b,2
2,c,3


If you require a specific type (or base type), you can specify this in `read_model`:

In [7]:
cereal.read_model("memory://example_model", supercls=ExampleModel).df

,foo,bar
0,a,1
1,b,2
2,c,3


Inspecting the path, you can see the file structure:

In [9]:
from fsspec.implementations.memory import MemoryFileSystem  # noqa

fs = MemoryFileSystem()

In [10]:
fs.glob("example_model/*")

['/example_model/3f084fb6b6024e8da54fb231e655c8ea',
 '/example_model/model.json',
 '/example_model/model.schema.json']